In [1]:
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split as split
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from psyke import Extractor
from psyke.utils.logic import pretty_theory
from psyke.extraction.hypercubic import Grid, FeatureRanker
from psyke.extraction.hypercubic.strategy import AdaptiveStrategy

In [2]:
# fetch dataset 
adult = fetch_ucirepo(id=2)
X = adult.data.features[['age', 'fnlwgt', 'education-num', 'race', 'sex',
                         'capital-gain', 'capital-loss', 'hours-per-week']]
y = adult.data.targets

l = list(np.unique(X.race))
y.income = y.income.apply(lambda x: x[:-1] if x[-1] == '.' else x)
X.sex = X.sex.apply(lambda x: 0 if x == 'Male' else 1)
X.race = X.race.apply(lambda x: l.index(x))

scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)

normalization = {key: (m, s) for key, m, s in zip(X.columns, scaler.mean_, scaler.scale_)}

data = X.join(y)

In [3]:
def remove(l):
    for i in l:
        if 'sex' in i:
            del i['sex']
        if 'race' in i:
            del i['race']

In [4]:
cols = ['accuracy', 'fidelity', 'bias', 'treatment', 'FTA', 'DI', 'CF', 'comp_i', 'comp_g']

results = {
    'GB': {c: [] for c in cols},
    'KNN': {c: [] for c in cols},
    'SVM': {c: [] for c in cols}
}

for seed in [123, 345, 874, 128, 963, 741, 159, 853, 698, 742]:
    #print(f"******************{seed}******************")

    train, test = split(data, test_size=.5, random_state=seed)

    # conditions for test individuals (males and females)
    M = test['sex'] == min(test['sex'])
    F = test['sex'] == max(test['sex'])

    # test instances w/out protected attributes and output
    actionable_test = test.drop(['sex', 'race', 'income'], axis=1)
    
    gb = GB(n_estimators=200, learning_rate=0.2, random_state=seed
           ).fit(train.iloc[:, :-1], train.iloc[:, -1])
    #print(f"GB accuracy = {gb.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['GB']['accuracy'].append(gb.score(test.iloc[:, :-1], test.iloc[:, -1]))

    knn = KNN(n_neighbors=150).fit(train.iloc[:, :-1], train.iloc[:, -1])
    #print(f"KNN accuracy = {knn.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['KNN']['accuracy'].append(knn.score(test.iloc[:, :-1], test.iloc[:, -1]))

    svc = SVC(C=0.85, random_state=seed).fit(train.iloc[:, :-1], train.iloc[:, -1])
    #print(f"SVM accuracy = {svc.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['SVM']['accuracy'].append(svc.score(test.iloc[:, :-1], test.iloc[:, -1]))

    for i, (name, predictor) in enumerate(zip(results.keys(), [gb, knn, svc])):
        extractor = Extractor.gridex(predictor, Grid(1, AdaptiveStrategy(
            FeatureRanker(data.columns[:-1]).fit(predictor, data.iloc[:, :-1]).rankings(), 
            [(0.15, 2), (0.9, 3)])), threshold=.1, min_examples=100, normalization=normalization)
        theory = extractor.extract(train)
        fid = extractor.accuracy(test, predictor)
        #print(f'\nExtractor performance ({extractor.n_rules} rules): Fidelity = {fid:.2f}')
        results[name]['fidelity'].append(fid)
        
        # conditions for test individuals (positive and negative)
        preds = predictor.predict(test.iloc[:, :-1])
        pos = preds == '>50K'
        neg = preds == '<=50K'
        
        groups = {
            'ALL': M | F,
            'M+': M & pos,
            'M-': M & neg,
            'F+': F & pos,
            'F-': F & neg,
            '+': pos,
            '-': neg
        }
        
        mi, ma = min(test.sex), max(test.sex)
        mic = test.sex == mi
        mac = test.sex == ma
        testc = test.copy()
        testc.sex[mic] = ma
        testc.sex[mac] = mi
        predsc = predictor.predict(testc.iloc[:, :-1])
        results[name]['CF'].append(sum(preds == predsc) / len(preds))

        results[name]['DI'].append(
            (len(test[groups['F+']]) / len(test[F])) / (len(test[groups['M+']]) / len(test[M]))
        )
        
        closest = test.iloc[BallTree(actionable_test, p=2
                                    ).query(actionable_test, k=2)[1][:, -1]]
        predsc = predictor.predict(closest.iloc[:, :-1])
        results[name]['FTA'].append(sum(preds == predsc) / len(preds))

        res = [extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1]
               for _, r in test.iterrows()]
        bias = np.array(
            [('sex' in r and r['sex'] != []) or ('race' in r and r['race'] != []) for r in res]
        )

        for n, g in groups.items():
            try:
                b = sum(bias[g]) / len(bias[g])
            except ZeroDivisionError:
                b = 0
            results[name]['bias'].append((n, b))
            
        pairs = []
        for c in [pos, neg]:
            closest = test[c].iloc[BallTree(actionable_test[c], p=2
                                           ).query(actionable_test[c], k=2)[1][:, -1]]
            pairs += [
                extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1] != 
                extractor.predict_counter(dict(c[:-1]), verbose=False, only_first=True)[1][0][1]
                for (_, r), (_, c) in zip(test[c].iterrows(), closest.iterrows())
            ]
        results[name]['treatment'].append(('ALL', sum(pairs) / len(pairs)))
        
        results[name]['comp_i'].append(1 - (sum(bias[pos] | np.array(pairs)[pos]) + 
                                            sum(bias[neg] | np.array(pairs)[neg])) / len(bias))
        gf = []
        for n, (g1, g2) in zip(['MF+', 'MF-', 'FM+', 'FM-'], 
                               [('M+', 'F+'), ('M-', 'F-'), ('F+', 'M+'), ('F-', 'M-')]):
            g1 = groups[g1]
            g2 = groups[g2]
            closest = actionable_test[g2].iloc[
                BallTree(actionable_test[g2], p=2).query(actionable_test[g1], k=1)[1][:, -1]]
            pairs = [
                extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1] != 
                extractor.predict_counter(dict(c[:-1]), verbose=False, only_first=True)[1][0][1]
                for (_, r), (_, c) in zip(test[g1].iterrows(), closest.iterrows())
            ]
            results[name]['treatment'].append((n, sum(pairs) / len(pairs)))
            gf.append(1 - sum(bias[g1] | pairs) / len(pairs))
        results[name]['comp_g'].append(np.min(gf))

In [5]:
for r in results:
    print(r)
    print(f"Accuracy: {np.mean(results[r]['accuracy']):.2f} +/- {np.std(results[r]['accuracy']):.2f}")
    print(f"Fidelity: {np.mean(results[r]['fidelity']):.2f} +/- {np.std(results[r]['fidelity']):.2f}")
    
    print(f"FTA: {np.mean(results[r]['FTA']):.2f} +/- {np.std(results[r]['FTA']):.2f}")
    print(f"CF: {np.mean(results[r]['CF']):.2f} +/- {np.std(results[r]['CF']):.2f}")
    bias = results[r]['bias']
    print(f"Individual bias: {np.mean([b[1] for b in bias if b[0] == 'ALL']):.2f} +/- {np.std([b[1] for b in bias if b[0] == 'ALL']):.2f}")
    treat = results[r]['treatment']
    print(f"Individual treatment: {np.mean([t[1] for t in treat if t[0] == 'ALL']):.2f} +/- {np.std([t[1] for t in treat if t[0] == 'ALL']):.2f}")
    print(f"Individual comprehensive fairness: {np.mean(results[r]['comp_i']):.2f} +/- {np.std(results[r]['comp_i']):.2f}")

    print(f"DI: {np.mean(results[r]['DI']):.2f} +/- {np.std(results[r]['DI']):.2f}")
    print("Group bias")
    
    gb = {}
    cat = ['M+', 'F+', 'M-', 'F-']
    for g in cat:
        gb[g] = [b[1] for b in bias if b[0] == g]
        print(f"     group {g}: {np.mean(gb[g]):.2f} +/- {np.std(gb[g]):.2f}")
    comb = [np.mean(g) for g in gb.values()]
    mi = np.array([min(gb[k][i] for k in cat) for i in range(len(gb['M+']))])
    ma = np.array([max(gb[k][i] for k in cat) for i in range(len(gb['M+']))])
    print(f"     combined: {np.mean(ma - mi):.2f} +/- {np.std(ma - mi):.2f}")
    
    print("Group treatment")
    for g in ['MF+', 'MF-', 'FM+', 'FM-']:
        print(f"     group {g}: {np.mean([t[1] for t in treat if t[0] == g]):.2f} +/- {np.std([t[1] for t in treat if t[0] == g]):.2f}")
    
    print(f"Group comprehensive fairness: {np.mean(results[r]['comp_g']):.2f} +/- {np.std(results[r]['comp_g']):.2f}")
    print()

GB
Accuracy: 0.85 +/- 0.00
Fidelity: 0.88 +/- 0.00
FTA: 0.92 +/- 0.00
CF: 0.89 +/- 0.00
Individual bias: 0.31 +/- 0.00
Individual treatment: 0.38 +/- 0.00
Individual comprehensive fairness: 0.43 +/- 0.01
DI: 0.17 +/- 0.00
Group bias
     group M+: 0.33 +/- 0.00
     group F+: 0.42 +/- 0.02
     group M-: 0.07 +/- 0.03
     group F-: 0.70 +/- 0.05
     combined: 0.63 +/- 0.08
Group treatment
     group MF+: 1.00 +/- 0.00
     group MF-: 1.00 +/- 0.00
     group FM+: 1.00 +/- 0.00
     group FM-: 1.00 +/- 0.00
Group comprehensive fairness: 0.00 +/- 0.00

KNN
Accuracy: 0.82 +/- 0.00
Fidelity: 0.90 +/- 0.00
FTA: 0.94 +/- 0.00
CF: 0.87 +/- 0.00
Individual bias: 0.30 +/- 0.04
Individual treatment: 0.40 +/- 0.00
Individual comprehensive fairness: 0.42 +/- 0.03
DI: 0.08 +/- 0.01
Group bias
     group M+: 0.03 +/- 0.00
     group F+: 0.36 +/- 0.13
     group M-: 0.00 +/- 0.00
     group F-: 0.91 +/- 0.12
     combined: 0.91 +/- 0.12
Group treatment
     group MF+: 1.00 +/- 0.00
     group MF-: 